In [ ]:
%reset -f
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [15]:
from tqdm import tqdm
# from tqdm.notebook import tqdm as tqdm
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset, ConcatDataset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from tsai.all import *

torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False
torchaudio.set_audio_backend('soundfile')
import os
import random
from pathlib import Path

import matplotlib.pyplot as plt
import psutil
import requests

from utils import *
from wingbeat_datasets import *
from wingbeat_models import *

print(f'Total RAM      : {bytes2GB(psutil.virtual_memory().total):5.2f} GB')
print(f'Available RAM  : {bytes2GB(psutil.virtual_memory().available):5.2f} GB\n')

Total RAM      : 31.21 GB
Available RAM  : 19.56 GB



In [16]:
num_epochs = 35
batch_size = 32
batch_size_val = batch_size * 2
validation_split = .2
shuffle_dataset = True
num_workers = psutil.cpu_count()
random_seed= 42
setting = 'psdl2'

In [17]:
dmel1 = clean_wingbeatdataset(name="Melanogaster_RL/Y", custom_label=0, transform=transforms.Compose([FilterWingbeat(setting='bandpass'), TransformWingbeat(setting=setting)]))
dmel2 = clean_wingbeatdataset(name="Melanogaster_RL/Z", custom_label=0, transform=transforms.Compose([FilterWingbeat(setting='bandpass'), TransformWingbeat(setting=setting)]))
dsuz1 = clean_wingbeatdataset(name="Suzukii_RL/L", custom_label=1,transform=transforms.Compose([FilterWingbeat(setting='bandpass'), TransformWingbeat(setting=setting)]))
dsuz2 = clean_wingbeatdataset(name="Suzukii_RL/R", custom_label=1,transform=transforms.Compose([FilterWingbeat(setting='bandpass'), TransformWingbeat(setting=setting)]))

# dmel1 = WingbeatsDataset("Melanogaster_RL/Y", custom_label=0, transform=transforms.Compose([FilterWingbeat(setting='bandpass')]))
# dmel2 = WingbeatsDataset("Melanogaster_RL/Z", custom_label=0, transform=transforms.Compose([FilterWingbeat(setting='bandpass')]))
# dsuz1 = WingbeatsDataset("Suzukii_RL/L", custom_label=1,transform=transforms.Compose([FilterWingbeat(setting='bandpass')]))
# dsuz2 = WingbeatsDataset("Suzukii_RL/R", custom_label=1,transform=transforms.Compose([FilterWingbeat(setting='bandpass')]))
# dmel1 = WingbeatsDataset("Melanogaster_RL/Y", custom_label=0, transform=transforms.Compose([FilterWingbeat(setting='bandpass'),TransformWingbeat(setting='psdl1')]))
# dmel2 = WingbeatsDataset("Melanogaster_RL/Z", custom_label=0, transform=transforms.Compose([FilterWingbeat(setting='bandpass'),TransformWingbeat(setting='psdl1')]))
# dsuz1 = WingbeatsDataset("Suzukii_RL/L", custom_label=1,transform=transforms.Compose([FilterWingbeat(setting='bandpass'),TransformWingbeat(setting='psdl1')]))
# dsuz2 = WingbeatsDataset("Suzukii_RL/R", custom_label=1,transform=transforms.Compose([FilterWingbeat(setting='bandpass'),TransformWingbeat(setting='psdl1')]))

Found 29002 in dataset: Melanogaster_RL/Y, and 1 label(s): ['Y']
Label(s) changed to 0


  0%|          | 0/454 [00:00<?, ?it/s]

Found 29002 in dataset: Melanogaster_RL/Y, and 1 label(s): ['Y']
Label(s) changed to 


  0%|          | 0/387 [00:00<?, ?it/s]

Found 24763 in dataset: Melanogaster_RL/Z, and 1 label(s): ['Z']
Label(s) changed to 0
Found 24763 in dataset: Melanogaster_RL/Z, and 1 label(s): ['Z']
Label(s) changed to 


  0%|          | 0/343 [00:00<?, ?it/s]

Found 21940 in dataset: Suzukii_RL/L, and 1 label(s): ['L']
Label(s) changed to 1
Found 21940 in dataset: Suzukii_RL/L, and 1 label(s): ['L']
Label(s) changed to 


  0%|          | 0/225 [00:00<?, ?it/s]

Found 14348 in dataset: Suzukii_RL/R, and 1 label(s): ['R']
Label(s) changed to 1
Found 14348 in dataset: Suzukii_RL/R, and 1 label(s): ['R']
Label(s) changed to 


100%|██████████| 225/225 [00:44<00:00,  5.05it/s]


In [6]:
transformed_dataset = ConcatDataset([dmel1, dsuz1])

train_size = int(0.8 * len(transformed_dataset))
valid_size = len(transformed_dataset) - train_size
train_dataset, valid_dataset = torch.utils.data.random_split(transformed_dataset, [train_size, valid_size])
test_dataset = ConcatDataset([dmel2, dsuz2])

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size_val, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size_val, num_workers=num_workers)

In [8]:
if setting.startswith('psd'):
    model = Conv1dNetPSD()
else:
    model = Conv1dNetRAW()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, verbose=True)
early_stopping = EarlyStopping(patience=7, verbose=False)
# print(model)

In [9]:
# from sklearn import preprocessing
# import itertools

# le = preprocessing.LabelEncoder()
# all_labels = [transformed_dataset.datasets[i].labels for i in range(len(transformed_dataset.datasets))]
# all_labels = list(itertools.chain.from_iterable(all_labels))
# le.fit(all_labels)

## Training

In [10]:
# Choosing whether to train on a gpu
train_on_gpu = torch.cuda.is_available()
print(f'Train on gpu: {train_on_gpu}')# Number of gpus
model = model.to('cuda', dtype=torch.float)

Train on gpu: True


In [11]:
# Model training
for epoch in range(num_epochs):
    # Going through the training set
    correct_train = 0
    model.train()
    for x_batch,y_batch,path_batch,idx_batch in tqdm(train_dataloader, desc='Training..\t'):        

        y_batch = torch.as_tensor(y_batch).type(torch.LongTensor)
        x_batch,y_batch = x_batch.cuda(), y_batch.cuda()
        
        optimizer.zero_grad()
        pred = model(x_batch)
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()
        correct_train += (pred.argmax(axis=1) == y_batch).float().sum().item()

    train_accuracy = correct_train / (len(train_dataloader)*batch_size) * 100.
    # Going through the validation set
    correct_valid = 0
    model.eval()
    for x_batch,y_batch,path_batch,idx_batch in tqdm(valid_dataloader, desc='Validating..\t'):
        
        y_batch = torch.as_tensor(y_batch).type(torch.LongTensor)
        x_batch,y_batch = x_batch.cuda(), y_batch.cuda()

        pred = model(x_batch)
        val_loss = criterion(pred, y_batch)
        correct_valid += (pred.argmax(axis=1) == y_batch).float().sum().item()
    valid_accuracy = correct_valid / (len(valid_dataloader)*batch_size_val) * 100.
    scheduler.step(val_loss)
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

        # Printing results
    print(f"Epoch {epoch}: train_acc: {train_accuracy:.2f}% loss: {loss:.3f},  val_loss: {val_loss:.3f} val_acc: {valid_accuracy:.2f}%")


Training..:   0%|          | 0/689 [00:00<?, ?it/s]

Epoch 0: train_acc: 83.66% loss: 0.528,  val_loss: 3.213 val_acc: 46.59%


Training..:   0%|          | 0/689 [00:00<?, ?it/s]

Epoch 1: train_acc: 87.50% loss: 0.425,  val_loss: 22.115 val_acc: 46.59%


Training..:   0%|          | 0/689 [00:00<?, ?it/s]

...

Epoch 16: train_acc: 97.21% loss: 0.041,  val_loss: 0.117 val_acc: 94.68%


Training..:   0%|          | 0/689 [00:00<?, ?it/s]

Epoch 17: train_acc: 97.23% loss: 0.033,  val_loss: 0.096 val_acc: 94.63%


Validating..: 100%|██████████| 87/87 [00:01<00:00, 83.45it/s] 

Early stopping


## Testing

In [13]:
correct_test = 0
model.eval()
for x_batch,y_batch,path_batch,idx_batch in test_dataloader:

    y_batch = torch.as_tensor(y_batch).type(torch.LongTensor)
    x_batch,y_batch = x_batch.cuda(), y_batch.cuda()

    pred = model(x_batch)
    val_loss = criterion(pred, y_batch)
    correct_test += (pred.argmax(axis=1) == y_batch).float().sum().item()
test_accuracy = correct_test / (len(test_dataloader)*batch_size_val) * 100.
print(test_accuracy)

91.31844380403457
